# BERT Example

In [1]:
!pip install scikit-learn datasets transformers evaluate numpy huggingface_hub torch

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0" 
os.environ["NCCL_SHM_DISABLE"] = "1" 


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import evaluate
import numpy as np
from datasets import load_dataset
from huggingface_hub import interpreter_login
from transformers import (AutoModelForSequenceClassification, AutoTokenizer,
                          DataCollatorWithPadding, Trainer, TrainingArguments,
                          pipeline)

# get an access token from https://huggingface.co/settings/tokens
interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|



## 1. Text Classification

## 1.1 Load Data

Text classification is a common NLP task that assigns a label or class to text. Some of the largest companies run text classification in production for a wide range of practical applications. One of the most popular forms of text classification is sentiment analysis, which assigns a label like positive, negative, or neutral to a sequence of text.

We use the following IMDb dataset to demonstrate text classification with BERT. The dataset contains 50,000 movie reviews, which are labeled as positive or negative. We will train a BERT model to classify the sentiment of the reviews.

There are two fields in this dataset:

text: the movie review text.
label: a value that is either 0 for a negative review or 1 for a positive review.

In [3]:
# Load the dataset
imdb = load_dataset('imdb')
imdb["test"][0]

{'text': 'I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn\'t match the background, and painfully one-dimensional characters cannot be overcome with a \'sci-fi\' setting. (I\'m sure there are those of you out there who think Babylon 5 is good sci-fi TV. It\'s not. It\'s clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It\'s really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it\'s rubbish as 

## 1.2 Preprocess: Tokenization

- Since BERT requires a specific input format, we need to preprocess the text data. The first step is to tokenize the text data. Tokenization is the process of splitting the text into individual words or subwords. 
- The BERT has a limit of 512 tokens per input, so we need to truncate or pad the input text to fit this limit.

In [4]:
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

# Create a preprocessing function to tokenize text and truncate sequences to be no longer than DistilBERT’s maximum input length:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

# Apply the preprocessing function to the dataset
tokenized_imdb = imdb.map(preprocess_function, batched=True)

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [5]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## 1.3 Training

In [6]:
# Include a metric to compute accuracy during training
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

# Sentiment labels
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

# Load the model
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# Train the model
training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_imdb["train"],
    eval_dataset=tokenized_imdb["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.223600,0.226367,0.911240
2,0.149000,0.232400,0.931520


TrainOutput(global_step=3126, training_loss=0.20540183244877264, metrics={'train_runtime': 1277.8668, 'train_samples_per_second': 39.128, 'train_steps_per_second': 2.446, 'total_flos': 6556904415524352.0, 'train_loss': 0.20540183244877264, 'epoch': 2.0})

## Sentiment Classification

After fine-tuning the BERT model on the IMDb dataset, we can use the model to classify the sentiment of movie reviews. We will use the following example to demonstrate sentiment classification:

In [7]:
text = "This was a masterpiece. Not completely faithful to the books, but enthralling from beginning to end. Might be my favorite of the three."

classifier = pipeline("sentiment-analysis", model="Morrisovo/my_awesome_model", device=0)
classifier(text)

config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Device set to use mps:0


[{'label': 'POSITIVE', 'score': 0.9976010918617249}]

In [8]:
text = "This was a disappointment. Strayed too far from the books, and failed to captivate from beginning to end. Easily my least favorite of the three."

classifier = pipeline("sentiment-analysis", model="Morrisovo/my_awesome_model", device=0)
classifier(text)

Device set to use mps:0


[{'label': 'NEGATIVE', 'score': 0.9928188920021057}]